# Numerical Data - Cleanup

<br>
**Author** Mary Letey <br>
**Date** 4/16/18 <br>
**File** megaset.ipynb <br>
**Purpose** Creating a mega-dataset aggregating all previous data sets, sorted by date for the last 5 years

<br><br>

In [3]:
# Imports
import numpy as np
import pandas as pd
import datetime
from dateutil import parser
import matplotlib.pylab as plt
%matplotlib inline

In [13]:
# -- dell seeking alpha
df = pd.read_csv("../data/scraper/dell_5yrs_seekingalpha.csv");
dates = list(df["date"]);

newdelldates = []
for i in range(0,len(dates)):
    mystr = dates[i]
    newdelldates.append(parser.parse(mystr))
    
delldates1 = newdelldates;


dftotal = df;
dftotal["date"] = delldates1;

dftotal.sort_values(df.columns[1])




,title,date,body
19,A Look Into Former Berkshire Hathaway Equity I...,2013-01-04 10:01:00,My favorite place to look for investment ideas...
7,The Cyclical And Secular Risks Of The PC Industry,2013-01-08 18:13:00,The PC industry is affected by both cyclical t...
23,Apple: Cheaper Phone Means Cheaper Stock?,2013-01-16 00:00:00,"Apple Inc. (NASDAQ:AAPL), longtime maker of Ma..."
8,The Best Way To Invest In Technology,2013-01-18 00:00:00,My Seeking Alpha fellow commentator Paulo Sant...
9,"Microsoft: 3.1% Dividend Yield, Good Upside, L...",2013-01-22 00:00:00,After looking into the valuation of Dell (NASD...
5,Seagate Technology: This Buyout Bust Still Mad...,2013-01-25 00:00:00,With the buyout talks of Dell heating up and t...
21,A 'Superabundance Of Capital' ... But Not For ...,2013-02-12 14:57:00,"""Bain & Company, the consultancy, forecasts a ..."
18,LSI: A Debt Free $7 Tech Stock,2013-02-14 00:00:00,LSI Corporation (NASDAQ:LSI-OLD) is a company ...
6,5 Thoughts On The Berkshire/3G - Heinz Deal,2013-02-16 00:00:00,When you combine the most oft discussed invest...
17,Building A Magic Formula Dividend Portfolio,2013-02-25 17:51:00,"With low interest rates in the bond market, ma..."


In [21]:
df = pd.read_csv("../data/scraper/dell_5yrs_wsj.csv");
dates = list(df["date"]);
titles = list(df["date"]);
del dates[15]
del titles[15]
df.head()



2013 年 4 月 5 日 18:54 JST


,title,date,body
0,Lenovo’s Diplomatic Response to Dell Buyout,"Feb 6, 2013 5:44 am ET",BLOOMBERGDell Inc.’s $24.4 billion deal to tak...
1,Blackstone Ends Pursuit of Dell,"Updated April 22, 2013 12:50 p.m. ET",Blackstone Group LP has ended its pursuit of D...
2,Dell to Sell Itself for $24.4 Billion,"Updated Feb. 5, 2013 8:50 p.m. ET",Dell Inc. on Tuesday struck a $24.4 billion de...
3,Dell Approves $91.1 Million in Retention Bonuses,"Updated April 23, 2013 10:00 p.m. ET",Dell Inc. disclosed Tuesday that its board app...
4,Dell Deal Negotiator Walks Fine Line,"May 6, 2013 6:57 p.m. ET","Last August, Alex Mandl was hosting dinner gue..."


In [91]:
a = [1,2,3,4,5,6,7,8,9]

visited = 0
index = 0
l = len(a)
print("Total = ",l)
while visited < l-(3-1):
    print(a)
    print("i=",index)
    if a[index]%3 == 0:
        del a[index]
#         index -= 1
    index += 1
    visited += 1
    print("we have vistited",visited,"elements")

print(a)
    
    


Total =  9
[1, 2, 3, 4, 5, 6, 7, 8, 9]
i= 0
we have vistited 1 elements
[1, 2, 3, 4, 5, 6, 7, 8, 9]
i= 1
we have vistited 2 elements
[1, 2, 3, 4, 5, 6, 7, 8, 9]
i= 2
we have vistited 3 elements
[1, 2, 4, 5, 6, 7, 8, 9]
i= 3
we have vistited 4 elements
[1, 2, 4, 5, 6, 7, 8, 9]
i= 4
we have vistited 5 elements
[1, 2, 4, 5, 7, 8, 9]
i= 5
we have vistited 6 elements
[1, 2, 4, 5, 7, 8, 9]
i= 6
we have vistited 7 elements
[1, 2, 4, 5, 7, 8]
